# Lasso Regression Model

This code tests lags for sentiment to test which ones are most correlated with 

In [10]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
df = pd.read_pickle('./IntermediatePickles/model_df.pkl')

# Make Data Stationary
by taking the difference (we are interested in rate-of-change anyway)

In [11]:
df['GDP_interp'] = df.GDP_interp.diff()
df['sentiment'] = df.sentiment.diff()

# Create Lags

Shift GDP by one and two quarters (data is quarterly)

Shift sentiment by 36 months

In [12]:
df['GDP_interp_L4'] = df.GDP_interp.shift(4)
#df['GDP_interp_L8'] = df.GDP_interp.shift(8)
for i in range(36):
    df['sentiment_L' + str(i)] = df.sentiment.shift(i)
df.dropna(inplace=True)
df.drop(columns='sentiment_L0',inplace=True)

# Define alpha

This will get increased until we only have one coefficient

In [13]:
alpha = 3

# Define pipeline
Lasso regression requires standard scaling

In [14]:
pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('lasso', Lasso(alpha = alpha,max_iter=100000))
])

# Define Cutoff
This is the point we will make predictions to test the accuracy of the model

In [15]:
cutoff = datetime.strptime('2014-09-01','%Y-%m-%d')

In [16]:
test_df = df.loc[df.chart_date > cutoff]
train_df = df.loc[df.chart_date <= cutoff]
X_train = train_df.drop(columns=['chart_date','GDP_interp'])
X_test = test_df.drop(columns=['chart_date','GDP_interp'])
y_train = pd.DataFrame(train_df.GDP_interp)
y_test = pd.DataFrame(test_df.GDP_interp)

# Fit Model

In [17]:
lasso_fit = pipe.fit(X_train,y_train)

# Test Model

In [18]:
pd.DataFrame(list(zip(list(X_train.columns),lasso_fit.named_steps.lasso.coef_)))

,0,1
0,sentiment,0.000000
1,GDP_interp_L4,24.512603
2,sentiment_L1,-0.000000
3,sentiment_L2,0.000000
4,sentiment_L3,0.000000
5,sentiment_L4,0.000000
6,sentiment_L5,-0.000000
7,sentiment_L6,-0.000000
8,sentiment_L7,0.000000
9,sentiment_L8,-0.000000


# Results
Sentiment with a 19-month lag is the most significant result